# Perform bottom line test for SMP-16-010

We will follow stat comm recommendations here: 

https://twiki.cern.ch/twiki/bin/viewauth/CMS/ScrecUnfolding

If we define $r$ to be the reconstructed MC, $R$ to be the response matrix, $g$ to be the generator-level truth, $u$ to be the unfolded reconstructed MC, and $\sigma$ to be the uncertainties, we compute:


- $\chi^2_{smeared} = |(r - Rg)/\sigma|^2$
- $\chi^2_{unfolded} = |(u - g)|^2$

## Set up the response matrices and vectors

In [1]:
import ROOT
ROOT.gSystem.Load("RooUnfold/libRooUnfold")
ROOT.gROOT.SetBatch()
import math

from ROOT import gRandom, TH1, cout, TH2, TLegend, TFile
from ROOT import RooUnfoldResponse
from ROOT import RooUnfold
from ROOT import RooUnfoldBayes
from ROOT import TCanvas
from ROOT import RooUnfoldSvd

extension = ''

mcfile = TFile('responses_rejec_tightgen_otherway_qcdmc_2dplots.root')
datafile = TFile('jetht_weighted_dataplots_otherway_rejec.root')

response = mcfile.Get('2d_response'+ extension)
responseSD = mcfile.Get('2d_response_softdrop' + extension)

truth = mcfile.Get('PFJet_pt_m_AK8Gen')
truthSD = mcfile.Get('PFJet_pt_m_AK8SDgen')


reco = mcfile.Get('PFJet_pt_m_AK8')
recoSD = mcfile.Get('PFJet_pt_m_AK8SD')

unfold = RooUnfoldBayes(response, reco, 4)    
unfoldSD = RooUnfoldBayes(responseSD, recoSD, 4)    

reco_unfolded = unfold.Vreco()
reco_det = unfold.Vmeasured()
response = unfold.response()
mresponse = response.Mresponse()
truth = response.Vtruth()
truth_folded = ROOT.TVectorD(truth)
truth_folded *= mresponse
errs = unfold.ErecoV(2)




reco_unfoldedSD = unfoldSD.Vreco()
reco_detSD = unfoldSD.Vmeasured()
responseSD = unfoldSD.response()
mresponseSD = responseSD.Mresponse()
truthSD = responseSD.Vtruth()
truth_foldedSD = ROOT.TVectorD(truthSD)
truth_foldedSD *= mresponseSD
errsSD = unfoldSD.ErecoV(2)




Welcome to JupyROOT 6.12/04


## Compute $\chi^2$ values for ungroomed case:

In [2]:

for ival in xrange( reco_det.GetNrows() ):
    reco_det[ival] -= truth_folded[ival]

for ival in xrange( reco_unfolded.GetNrows() ):
    reco_unfolded[ival] -= truth[ival]
    if errs[ival] > 0.0 : 
        reco_unfolded[ival] /= errs[ival]
        
        
print '-----------------'
print 'Ungroomed:'
print 'Chi2_folded   = ',  reco_det.Norm1() 
print 'Chi2_unfolded = ', reco_unfolded.Norm1() 
print '-----------------'



-----------------
Ungroomed:
Chi2_folded   =  1170.85229835
Chi2_unfolded =  374.010680338
-----------------


## Compute $\chi^2$ values for groomed case:

In [3]:



for ival in xrange( reco_detSD.GetNrows() ):
    reco_detSD[ival] -= truth_foldedSD[ival]

for ival in xrange( reco_unfoldedSD.GetNrows() ):
    reco_unfoldedSD[ival] -= truthSD[ival]
    if errsSD[ival] > 0.0:
        reco_unfoldedSD[ival] /= errsSD[ival]
print 'Groomed:'
print 'Chi2_folded   = ',  reco_detSD.Norm1()
print 'Chi2_unfolded = ', reco_unfoldedSD.Norm1()




Groomed:
Chi2_folded   =  710.666554867
Chi2_unfolded =  207.491901169
